# Install library

In [ ]:
!pip install selenium

# Import library

In [9]:
import pandas as pd
import re
import time
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdraiver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
today = datetime.today().strftime("%Y-%m-%d")
from selenium import webdriver

# Scraper list

<b>DONE</b>:
1. republica: populer (https://republika.co.id/kanal/news)
2. kompas: https://www.kompas.com/, ada tren dan populer (10 terpopuler di halaman depan). Bisa scrape yg tren berdasarkan tanggal (cth: https://www.kompas.com/tren/search/2020-09-09)
3. tirto: populer hanya 4 (https://tirto.id/)
4. kumparan: tidak ada populer, adanya trending (https://kumparan.com/trending)
5. detik: berita terpopuler (https://www.detik.com/terpopuler?tag_from=wp_cb_mostPopular_more)
6. tribunnews: populer (https://www.tribunnews.com/populer)
7. liputan6: populer (https://www.liputan6.com/indeks/terpopuler)
8. merdeka.com: trending (https://www.merdeka.com/trending/)
9. kapanlagi.com: trending (https://www.kapanlagi.com/trending/)
10. okezone.com: terpopuler (https://www.okezone.com/)
11. tempo.co: populer (https://www.tempo.co/populer)
12. viva.co.id: populer (https://www.viva.co.id/)
13. suara.com: populer (https://suara.com/)
14. jpnn.com: populer (https://www.jpnn.com/populer)
15. sindonews.com: populer (https://www.sindonews.com/)
16. jawapos.com: trending (https://www.jawapos.com/)
17. antaranews.com: populer (https://www.antaranews.com/#tab-popular)
18. inilah.com: populer (https://inilah.com/)

<b>UNDONE</b>:
19. mayangkaranews.com: trending (https://mayangkaranews.com/) - berita blitar
20. bola.net (ttg bola): trending (https://www.bola.net/trending/). Bisa scrape yg tren berdasarkan tanggal (cth: https://www.bola.net/trending/2020/09/12/)
21. uzone.co.id (ttg otomotif dan teknologi)
22. metrotvnews.com (headline news, berita dalam bentuk video)

# Republica

In [8]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://republika.co.id/kanal/news'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="populer_set_left"]/ul/*')))
arr_li = driver.find_elements_by_xpath('//div[@class="populer_set_left"]/ul/*')
df_populer = pd.DataFrame()
for li in arr_li:
    a = li.find_element_by_tag_name('a')
    title = a.text
    link = a.get_attribute('href')
    df_populer = df_populer.append({
        'title': title,
        'link': link,
        'scraped_at': datetime.now()
    }, ignore_index=True)
driver.close()

df_populer.to_csv(
    'scrape result/republica/{}_populer_republica.csv'.format(date), index=False)

# Kompas

## Populer

In [4]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.kompas.com/'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="most__wrap clearfix"]/*')))
arr_div_populer = driver.find_elements_by_xpath(
    '//div[@class="most__wrap clearfix"]/*')
df_populer = pd.DataFrame()
for d in arr_div_populer:
    rank = d.find_element_by_class_name('most__count').text
    link = d.find_element_by_tag_name('a').get_attribute('href')
    title = d.find_element_by_tag_name('h4').text
    read = d.find_element_by_class_name('most__read').text
    df_populer = df_populer.append({
        'rank': rank,
        'link': link,
        'title': title,
        'read_count': read,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_populer.to_csv(
    'scrape result/kompas/{}_populer_kompas.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


## Trending

In [7]:
scraped_date = '2020-10-12'

# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
df_trending = pd.DataFrame()
p = 1
while True:
    url = 'https://www.kompas.com/tren/search/{}/{}'.format(scraped_date, p)
    driver.get(url)
    time.sleep(3)
    arr_div = driver.find_elements_by_xpath('//div[@class="latest--news mt2 clearfix"]/*')
    if len(arr_div) <= 0:
        break
    else:
        for d in arr_div:
            a = d.find_element_by_tag_name('h3').find_element_by_tag_name('a')
            title = a.text
            link = a.get_attribute('href')
            timestamp = d.find_element_by_class_name('article__date').text
            df_trending = df_trending.append({
                'page': p,
                'title': title,
                'link': link,
                'timestamp': timestamp,
                'scraped_at': datetime.now()
            }, ignore_index=True)
        p += 1

df_trending.to_csv('scrape result/kompas/{}_trending_kompas.csv'.format(scraped_date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Tirto

In [13]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://tirto.id/'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="container mt-28 container900"]/div[13]/*')))
df_populer = pd.DataFrame()
for i in range(2, 4):
    arr_row = driver.find_elements_by_xpath(
        '//div[@class="container mt-28 container900"]/div[13]/div[{}]/*'.format(i))
    for r in arr_row:
        title = r.find_element_by_tag_name('h1').text
        link = r.find_element_by_tag_name('a').get_attribute('href')
        df_populer = df_populer.append({
            'title': title,
            'link': link,
            'scraped_at': datetime.now()
        }, ignore_index=True)
        
df_populer.to_csv('scrape result/tirto/{}_populer_tirto.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Kumparan

In [17]:
def scroll_down():
    """A method for scrolling the page."""
    last_height = driver.execute_script(
        "return document.body.scrollHeight")

    while True:
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        new_height = driver.execute_script(
            "return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [19]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://kumparan.com/trending'
driver.get(url)
time.sleep(3)
scroll_down()
arr_div_trending = driver.find_elements_by_xpath(
    '//div[@class="Viewweb__StyledView-p5eu6e-0 hWzdds"]/*')
print(len(arr_div_trending))
df_trending = pd.DataFrame()
for d in arr_div_trending:
    try:
        link = d.find_element_by_tag_name('a').get_attribute('href')
        slug = re.sub(r'\/berita-viral\/', '', link)
        api = 'https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22{}%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22c05aebe153dfd967395d35aa6f9516d6141475b2223b445d01248b79b5506223%22%7D%7D&cache-ttl=10'.format(
            slug)
        print(api)
        response = requests.get(api)
        data = json.loads(response.content)
        df_temp = pd.DataFrame.from_dict(
            data['data']).T.reset_index().drop(columns=['index'])
        df_trending = df_trending.append(df_temp, ignore_index=True)
    except:
        pass
df_trending.to_csv(
    'scrape result/kumparan/{}_trending_kumparan.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

114
https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/kumparannews/habib-rizieq-tolak-tes-swab-dari-petugas-yang-datang-ke-rumahnya-1udZrwaCS60%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22c05aebe153dfd967395d35aa6f9516d6141475b2223b445d01248b79b5506223%22%7D%7D&cache-ttl=10
https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/kumparannews/ketua-pa-212-cilacap-meninggal-karena-corona-ada-riwayat-kondangan-1udcjmXzOOk%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22c05aebe153dfd967395d35aa6f9516d6141475b2223b445d01248b79b5506223%22%7D%7D&cache-ttl=10
https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/kumparannews/anak-habib-idrus-yang-doakan-jokowi-umur-pendek-disebut-asn-ini-kata-bappenas-1udaYEk

https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/kumparannews/foto-sisa-mayat-letusan-gunung-vesuvius-ditemukan-di-pompeii-italia-1udQwaFFW11%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22c05aebe153dfd967395d35aa6f9516d6141475b2223b445d01248b79b5506223%22%7D%7D&cache-ttl=10
https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/paluposo/di-palu-baliho-bergambar-habib-rizieq-juga-diturunkan-1udcGVfUZHb%22%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%22c05aebe153dfd967395d35aa6f9516d6141475b2223b445d01248b79b5506223%22%7D%7D&cache-ttl=10
https://cdn-graphql-v4.kumparan.com/query?operationName=FindStoryBySlug&variables=%7B%22slug%22%3A%22https://kumparan.com/kumparanhits/dita-soedarjo-ditanya-kapan-menikah-oleh-netizen-cuma-tuhan-yesus-yang-tahu-1udKP6Epjup%22%7D&extension

# Detik

In [22]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
tags = ['detikcom', 'news', 'finance', 'hot', 'inet', 'sport', 'oto',
        'travel', 'sepakbola', 'food', 'health', 'wolipop']
df_populer = pd.DataFrame()
for t in tags:
    if t == 'detikcom':
        url = 'https://www.detik.com/terpopuler/'
    else:
        url = 'https://www.detik.com/terpopuler/{}'.format(t)
    driver.get(url)
    time.sleep(3)
    
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="grid-row list-content"]/*')))
    arr_div_populer = driver.find_elements_by_xpath(
        '//div[@class="grid-row list-content"]/*')
    last_update = driver.find_element_by_xpath(
        '//div[@class="page__indeks-info font-base-semibold"]').text
    for d in arr_div_populer:
        link = d.find_element_by_tag_name('a').get_attribute('href')
        title = d.find_element_by_tag_name('h3').text
        div_other_info = d.find_element_by_class_name('media__date')
        category = div_other_info.text
        timestamp = div_other_info.find_element_by_tag_name(
            'span').get_attribute('d-time')
        df_populer = df_populer.append({
            'link': link,
            'title': title,
            'category': re.sub(r'\|.*', '', category).strip(),
            'timestamp': timestamp,
            'last_update': last_update,
            'scraped_at': datetime.now()
        }, ignore_index=True)
df_populer = df_populer.drop_duplicates(subset='link')
df_populer.to_csv(
    'scrape result/detik/{}_populer_detik.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Tribunnews

In [24]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
arr_types = ['6h', '12h', '1d', '3d', '1w']
df_populer = pd.DataFrame()
for t in arr_types:
    url = 'https://www.tribunnews.com/populer?section=&type={}'.format(t)
    driver.get(url)
    time.sleep(3)
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="lsi pt10 pb10"]/ul/*')))
    arr_li_populer = driver.find_elements_by_xpath(
        '//div[@class="lsi pt10 pb10"]/ul/*')

    count = 1
    for li in arr_li_populer:
        rank = driver.find_element_by_xpath(
            '//div[@class="lsi pt10 pb10"]/ul/li[{}]/div/div[1]'.format(count)).text
        link = li.find_element_by_tag_name('a').get_attribute('href')
        title = li.find_element_by_tag_name('h3').text
        timestamp = li.find_element_by_tag_name('time').text
        df_populer = df_populer.append({
            'type': t,
            'rank': rank,
            'link': link,
            'title': title,
            'timestamp': timestamp,
            'scraped_at': datetime.now()
        }, ignore_index=True)
        count += 1
df_populer.to_csv(
    'scrape result/tribunnews/{}_populer_tribunnews.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Liputan6

In [28]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
arr_types = ['hari-ini', 'minggu-ini', 'bulan-ini', 'tahun-ini', 'selamanya']
df_populer = pd.DataFrame()
for t in arr_types:
    p = 1
    arr_web_title = []
    while True:
        url = 'https://www.liputan6.com/indeks/terpopuler/{}?page={}'.format(
            t, p)
        driver.get(url)
        time.sleep(3)
        
        web_title = driver.find_element_by_xpath('//header[@class="header-index-alt"]/h1').text
        print(web_title)
        if len(arr_web_title) > 0 and arr_web_title[-1] == web_title:
            break

        arr_web_title.append(web_title) 
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="articles--list articles--list_rows"]/*')))
        arr_div_artikel = driver.find_elements_by_xpath(
            '//div[@class="articles--list articles--list_rows"]/*')
        for d in arr_div_artikel:
            link = d.find_element_by_tag_name('a').get_attribute('href')
            header = d.find_element_by_tag_name('header')
            category = header.find_element_by_tag_name('a').text
            try:
                timestamp = header.find_element_by_tag_name(
                    'time').get_attribute('datetime')
            except:
                timestamp = None
            title = header.find_element_by_tag_name('h4').text
            summary = d.find_element_by_tag_name(
                'aside').find_element_by_tag_name('div').text
            df_populer = df_populer.append({
                'type': t,
                'page': p,
                'link': link,
                'category': category,
                'timestamp': timestamp,
                'title': title,
                'summary': summary
            }, ignore_index=True)
        p += 1
df_populer.to_csv('scrape result/liputan6/{}_populer_liputan6.csv'.format(date), index=False)
driver.close()

BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 1
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 2
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 3
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 4
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 5
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 6
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 7
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 8
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 9
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 10
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 11
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 12
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI - HALAMAN 13
BERITA TERPOPULER, FOTO DAN VIDEO - LIPUTAN6.COM - HARI INI 

# Merdeka.com

In [30]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
df_trending = pd.DataFrame()
index = 1
while True:
    url = 'https://www.merdeka.com/trending/index{}'.format(index)
    driver.get(url)
    time.sleep(3)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//div[@class="inner-content"]/ul/*')))
        arr_li = driver.find_elements_by_xpath(
            '//div[@class="inner-content"]/ul/*')
        for count in range(len(arr_li)):
            try:
                a = driver.find_element_by_xpath(
                    '//div[@class="inner-content"]/ul/li[{}]/div/h3/a'.format(count+1))
                title = a.text
                link = a.get_attribute('href')
                try:
                    summary = driver.find_element_by_xpath(
                        '//div[@class="inner-content"]/ul/li[{}]/div/div[2]/p'.format(count+1)).text
                except:
                    summary = None
                timestamp = driver.find_element_by_xpath(
                    '//div[@class="inner-content"]/ul/li[{}]/div/div[2]/span[2]'.format(count+1)).text
                df_trending = df_trending.append({
                    'title': title,
                    'link': link,
                    'summary': summary,
                    'timestamp': timestamp,
                    'scraped_at': datetime.now(),
                    'page': index
                }, ignore_index=True)

            except:
                try:
                    a = driver.find_element_by_xpath(
                        '//div[@class="inner-content"]/ul/li[{}]/div/div[2]/h3/a'.format(count+1))
                    title = a.text
                    link = a.get_attribute('href')
                    try:
                        summary = driver.find_element_by_xpath(
                            '//div[@class="inner-content"]/ul/li[{}]/div/div[2]/p'.format(count+1)).text
                    except:
                        summary = None
                    timestamp = driver.find_element_by_xpath(
                        '//div[@class="inner-content"]/ul/li[{}]/div/div[2]/span[2]'.format(count+1)).text
                    df_trending = df_trending.append({
                        'title': title,
                        'link': link,
                        'summary': summary,
                        'timestamp': timestamp,
                        'scraped_at': datetime.now(),
                        'page': index
                    }, ignore_index=True)
                except:
                    pass
    except:
        break
    index += 1
df_trending.to_csv('scrape result/merdeka/{}_trending_merdeka.csv'.format(date), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Kapanlagi.com

In [31]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
df_trending = pd.DataFrame()
index = 1
while True:
    if index <= 1:
        url = 'https://www.kapanlagi.com/trending/index.html'
    else:
        url = 'https://www.kapanlagi.com/trending/index{}.html'.format(index)
        headline_title = None
        headline_link = None

    driver.get(url)
    time.sleep(3)

    if index <= 1:
        a_headline = driver.find_element_by_xpath(
            '//h2[@class="title-tag"]/a')
        headline_title = a_headline.text
        headline_link = a_headline.get_attribute('href')
    
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, '//li[@id="tagli"]')))
        arr_li = driver.find_elements_by_xpath('//li[@id="tagli"]')
        for li in arr_li:
            a = li.find_element_by_tag_name('div').find_element_by_tag_name('a')
            link = a.get_attribute('href')
            title = a.text
            date = li.find_element_by_class_name('date').text
            df_trending = df_trending.append({
                'link': link,
                'title': title,
                'date': date,
                'scraped_at': datetime.now(),
                'page': index,
                'headline_title': headline_title,
                'headline_link': headline_link
            }, ignore_index=True)
    except:
        break
    index += 1
df_trending.to_csv('scrape result/kapanlagi/{}_trending_kapanlagi.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Okezone.com

In [49]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.okezone.com/'
driver.get(url)
time.sleep(3)

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="berita-pilihan"]')))
arr_div = driver.find_elements_by_xpath('//div[@class="berita-pilihan"]')
df_populer = pd.DataFrame()
for d in arr_div:
    a = d.find_element_by_class_name(
        'content-hardnews').find_element_by_tag_name('a')
#     title = a.text
    link = a.get_attribute('href')
#     timestamp = d.find_element_by_tag_name('span').text
#     category = d.find_element_by_class_name(
#         'desk-berita-pilihan').find_elements_by_tag_name('a')[1].text
    df_populer = df_populer.append({
#         'title': title,
        'link': link,
        'scraped_at': datetime.now()
#         'timestamp': timestamp,
#         'category': category
    }, ignore_index=True)

df_populer = df_populer.drop_duplicates(subset='link')
df_populer.to_csv(
    'scrape result/okezone/{}_populer_okezone.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Tempo.co

In [34]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
arr_types = ['6jam', '12jam', '1hari', '1minggu', '1bulan', '1tahun']
df_populer = pd.DataFrame()
for t in arr_types:
    url = 'https://www.tempo.co/populer/?tipe={}'.format(t)
    driver.get(url)
    time.sleep(3)
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="wrapper"]/*')))
    arr_li = driver.find_elements_by_xpath(
        '//ul[@class="wrapper"]/*')
    for li in arr_li:
        link = li.find_element_by_tag_name('a').get_attribute('href')
        title = li.find_element_by_tag_name('h2').text
        summary = li.find_element_by_tag_name('p').text
        timestamp = li.find_element_by_tag_name('span').text
        df_populer = df_populer.append({
            'type': t,
            'link': link,
            'title': title,
            'summary': summary,
            'timestamp': timestamp,
            'scraped_at': datetime.now()
        }, ignore_index=True)
df_populer.to_csv('scrape result/tempo/{}_populer_tempo.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Viva

In [36]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
df_populer = pd.DataFrame()
url = 'https://www.viva.co.id/'
driver.get(url)
time.sleep(3)

WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//div[@class="side-popular"]/ul/*')))
arr_li = driver.find_elements_by_xpath(
    '//div[@class="side-popular"]/ul/*')
for li in arr_li:
    try:
        a = li.find_element_by_tag_name('a')
        title = a.text
        link = a.get_attribute('href')
        df_populer = df_populer.append({
            'link': link,
            'title': title,
            'scraped_at': datetime.now()
        }, ignore_index=True)
    except:
        pass
df_populer.to_csv('scrape result/viva/{}_populer_viva.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Suara.com

In [38]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://suara.com/'
driver.get(url)
time.sleep(3)

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[3]/div[3]/article[2]/div/div[4]/div/div/ul/*')))
arr_li = driver.find_elements_by_xpath(
    '/html/body/div[3]/div[3]/article[2]/div/div[4]/div/div/ul/*')
df_populer = pd.DataFrame()
for li in arr_li:
    category_time = li.find_element_by_tag_name('span').text
    category = re.sub(r'\|.*', '', category_time).strip()
    timestamp = re.sub(r'.*\|', '', category_time).strip()
    h3 = li.find_element_by_tag_name('h3')
    link = h3.find_element_by_tag_name('a').get_attribute('href')
    title = h3.text
    df_populer = df_populer.append({
        'category': category,
        'time': timestamp,
        'link': link,
        'title': title,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_populer.to_csv('scrape result/suara/{}_populer_suara.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# jpnn.com

In [40]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.jpnn.com/populer'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//ul[@class="content-list"]/*')))
arr_li = driver.find_elements_by_xpath(
    '//ul[@class="content-list"]/*')
df_populer = pd.DataFrame()
for li in arr_li:
    category = li.find_element_by_tag_name('strong').text
    timestamp = li.find_element_by_class_name('silver').text
    a = li.find_element_by_tag_name('h1').find_element_by_tag_name('a')
    title = a.text
    link = a.get_attribute('href')
    summary = li.find_element_by_tag_name('p').text
    df_populer = df_populer.append({
        'category': category,
        'timestamp': timestamp,
        'title': title,
        'link': link,
        'summary': summary,
        'scraped_at': datetime.now()
    }, ignore_index=True)
    
# take terpopuler
a_most_popular = driver.find_element_by_xpath(
    '//*[@id="content-utama"]/div[1]/div[1]/div/div/div[1]/h1/a')
title_most_popular = a_most_popular.text
link_most_popular = a_most_popular.get_attribute('href')
category_most_popular = driver.find_element_by_xpath(
    '//*[@id="content-utama"]/div[1]/div[1]/div/div/div[1]/h6/a/strong').text
timestamp_most_popular = driver.find_element_by_xpath(
    '//*[@id="content-utama"]/div[1]/div[1]/div/div/div[1]/h6/a/span').text
summary_most_popular = driver.find_element_by_xpath(
    '//*[@id="content-utama"]/div[1]/div[1]/div/div/div[1]/p').text

df_populer['title_most_popular'] = title_most_popular
df_populer['link_most_popular'] = link_most_popular
df_populer['category_most_popular'] = category_most_popular
df_populer['timestamp_most_popular'] = timestamp_most_popular
df_populer['summary_most_popular'] = summary_most_popular

df_populer.to_csv('scrape result/jpnn/{}_populer_jpnn.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Sindonews.com

In [41]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.sindonews.com'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="populer"]/ul/*')))
arr_li = driver.find_elements_by_xpath('//div[@class="populer"]/ul/*')
df_populer = pd.DataFrame()
for li in arr_li:
    a = li.find_element_by_tag_name('a')
    title = a.text
    link = a.get_attribute('href')
    df_populer = df_populer.append({
        'title': title,
        'link': link,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_populer.to_csv('scrape result/sindonews/{}_populer_sindonews.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# jawapos.com

In [42]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.jawapos.com/'
driver.get(url)
time.sleep(3)

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//div[@class="trending-list"]/*')))
arr_div = driver.find_elements_by_xpath(
    '//div[@class="trending-list"]/*')
df_trending = pd.DataFrame()
for d in arr_div:
    a = d.find_element_by_tag_name('a')
    title = a.text
    link = a.get_attribute('href')
    df_trending = df_trending.append({
        'title': title,
        'link': link,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_trending.to_csv('scrape result/jawapos/{}_trending_jawapos.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Antaranews

In [43]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://www.antaranews.com/#tab-popular'
driver.get(url)
time.sleep(3)

WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="tab-popular"]/*')))
arr_article = driver.find_elements_by_xpath('//*[@id="tab-popular"]/*')
df_populer = pd.DataFrame()
for r in arr_article:
    a = r.find_element_by_tag_name('h3').find_element_by_tag_name('a')
    title = a.text
    link = a.get_attribute('href')
    df_populer = df_populer.append({
        'title': title,
        'link': link,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_populer.to_csv(
    'scrape result/antaranews/{}_populer_antaranews.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished


# Inilah.com

In [45]:
# insert your chrome driver here
driver = webdriver.Chrome(
    './chromedriver_linux64/chromedriver')
url = 'https://inilah.com/'
driver.get(url)
time.sleep(3)
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.XPATH, '//*[@id="populer"]/*')))
arr_div = driver.find_elements_by_xpath('//*[@id="populer"]/*')
df_populer = pd.DataFrame()
for d in arr_div:
    a = d.find_element_by_tag_name('a')
    title = a.find_element_by_class_name('pop-title').text
    link = a.get_attribute('href')
    rank = a.find_element_by_class_name('pop-count').text
    df_populer = df_populer.append({
        'title': title,
        'link': link,
        'rank': rank,
        'scraped_at': datetime.now()
    }, ignore_index=True)
df_populer.to_csv('scrape result/inilah/{}_populer_inilah.csv'.format(today), index=False)
print('Scraping is finished')
driver.close()

Scraping is finished
